# 15 - Warehouse Woes

https://adventofcode.com/2024/day/15


In [5]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";
import images from "../../utils/images.ts";

In [6]:
// Read Input

const file = await Deno.readTextFile("input-base.txt");

const [firstPart, secondPart] = file.split("\n\n");
const area = firstPart.split("\n").map((line) => line.split(""));
const moves = secondPart.split("\n").join("").split("").filter((move) => move);
[moves, area]
moves

[
  "<", "v", "v", ">", "^", "<", "v", "^", ">", "v", ">", "^",
  "v", "v", "^", "v", ">", "v", "<", ">", "v", "^", "v", "<",
  "v", "<", "^", "v", "v", "<", "<", "<", "^", ">", "<", "<",
  ">", "<", ">", ">", "v", "<", "v", "v", "v", "<", ">", "^",
  "v", "^", ">", "^", "<", "<", "<", ">", "<", "<", "v", "<",
  "<", "<", "v", "^", "v", "v", "^", "v", ">", "^", "v", "v",
  "v", "<", "<", "^", ">", "^", "v", "^", "^", ">", "<", "<",
  ">", ">", ">", "<", ">", "^", "<", "<", ">", "<", "^", "v",
  "v", "^", "^", "<",
  ... 600 more items
]

In [7]:
// Part 1 - What is the sum of all boxes' GPS coordinates?

const isEmpty = el => el === '.';

const getInitialPosition = (area: [][]) => {
    for (let i = 0; i < area.length; i++) {
        for (let j = 0; j < area[i].length; j++) {
            if (area[i][j] === '@') {
                return [i, j];
            }
        }
    }
}

const getDirection = (directionID: string) => {
    switch (directionID) {
        case '^':
            return [-1, 0];
        case 'v':
            return [1, 0];
        case '<':
            return [0, -1];
        case '>':
            return [0, 1];
    }
}

const nextEmptyPositionInDirection = (area: [][], direction: [], position: []) => {
    let [i, j] = position;

    while (true) {
        i += direction[0];
        j += direction[1];

        if (i < 0 || j < 0 || i >= area.length || j >= area[i].length) {
            return null;
        }

        switch (area[i][j]) {
            case '#':
                return null;
            case '.':
                return [i, j];
        }
    }
    return null;
}

const printArea = (area) => area.forEach((line) => console.log(line.join('')))

const getAreaPostMoves = (area: [][], moves: []) => {
    let currentPosition = getInitialPosition(area);
    moves.forEach((move, i) => {
        const direction = getDirection(move);
        let nextEmptyPosition = nextEmptyPositionInDirection(area, direction, currentPosition);
        // console.log(nextEmptyPosition, direction, currentPosition, move);
        if (nextEmptyPosition) {
            while (!(nextEmptyPosition[0] === currentPosition[0] && nextEmptyPosition[1] === currentPosition[1])) {
                const previousToNextEmptyPosition = [
                    nextEmptyPosition[0] - direction[0],
                    nextEmptyPosition[1] - direction[1]
                ]
                area[nextEmptyPosition[0]][nextEmptyPosition[1]] = area[previousToNextEmptyPosition[0]][previousToNextEmptyPosition[1]];
                area[previousToNextEmptyPosition[0]][previousToNextEmptyPosition[1]] = '.';
                nextEmptyPosition = [
                    previousToNextEmptyPosition[0],
                    previousToNextEmptyPosition[1]
                ]

            }
            currentPosition = [
                currentPosition[0] + direction[0],
                currentPosition[1] + direction[1]
            ];
        }
    });
    return area;
};

const getGPSSum = (area: []) => area.reduce((acc, line, i) => {
    return acc + line.reduce((acc, el, j) => {
        if (el === 'O') {
            return acc + 100 * i + j;
        }
        return acc;
    }, 0);
}, 0);

const currentArea = objects.deepCopy(area);
const finalArea = getAreaPostMoves(currentArea, moves);
printArea(finalArea);
getGPSSum(finalArea);

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########


10092

In [8]:
// Part 2 - What is the sum of all boxes' GPS coordinates with wider area ?

const checkIfCanMove = (area: [][], direction: [], position: []) => {
    const stackToMove = [[{ x: position[0], y: position[1] }]];
    let stackLevel = -1;
    while (true) {
        stackLevel += 1;
        const stack = stackToMove[stackLevel];
        let emptyPositions = 0;
        for (let i = 0; i < stack.length; i += 1) {
            const { x, y } = stack[i];
            const next = { x: x + direction[0], y: y + direction[1] };
            if (next.x < 0 || next.y < 0 || next.x >= area.length || next.y >= area[i].length) {
                return null;
            }
            switch (area[next.x][next.y]) {
                case '.':
                    emptyPositions += 1;
                    break;
                case '#':
                    return null
                case '[':
                    if (!stackToMove[stackLevel + 1]) stackToMove[stackLevel + 1] = [];
                    stackToMove[stackLevel + 1].push({ x: next.x, y: next.y });
                    if (direction[0]) stackToMove[stackLevel + 1].push({ x: next.x, y: next.y + 1 });
                    break;
                case ']':
                    if (!stackToMove[stackLevel + 1]) stackToMove[stackLevel + 1] = [];
                    stackToMove[stackLevel + 1].push({ x: next.x, y: next.y });
                    if (direction[0] && area[x][y]) stackToMove[stackLevel + 1].push({ x: next.x, y: next.y - 1 });
                    break;
            }
            if (stackToMove[stackLevel + 1]) {
                // remove dupes from [] on top of []
                stackToMove[stackLevel + 1] = stackToMove[stackLevel + 1].filter((el, i, arr) => {
                    return arr.findIndex((el2) => el2.x === el.x && el2.y === el.y) === i;
                });
            }
        }
        if (emptyPositions === stack.length) {
            return stackToMove;
        }
    }
}

const getAreaPostMovesV2 = (area: [][], moves: []) => {
    let currentPosition = getInitialPosition(area);

    moves.forEach((move, i) => {
        const direction = getDirection(move);
        const movableStack = checkIfCanMove(area, direction, currentPosition);
        if (movableStack) {
            const reversed = movableStack.reverse();
            reversed.forEach((stack) => {
                stack.forEach(({ x, y }) => {
                    const next = { x: x + direction[0], y: y + direction[1] };
                    area[next.x][next.y] = area[x][y];
                    area[x][y] = '.';
                });
            });
            currentPosition = [currentPosition[0] + direction[0], currentPosition[1] + direction[1]];
        }
    });
    return area;
};

const getGPSSumV2 = (area: []) => area.reduce((acc, line, i) => {
    return acc + line.reduce((acc, el, j) => {
        if (el === '[') {
            return acc + 100 * i + j;
        }
        return acc;
    }, 0);
}, 0);

const getWiderArea = (area: [][]) => {
    const widerArea = [];
    area.forEach((line, i) => {
        const widerRow = [];
        line.forEach((el, j) => {
            switch (el) {
                case '#':
                    widerRow.push('#');
                    widerRow.push('#');
                    break;
                case '.':
                    widerRow.push('.');
                    widerRow.push('.');
                    break;
                case 'O':
                    widerRow.push('[');
                    widerRow.push(']');
                    break;
                case '@':
                    widerRow.push('@');
                    widerRow.push('.');
                    break;
            }
        });
        widerArea.push(widerRow);
    });
    return widerArea;
}

const widerArea = getWiderArea(objects.deepCopy(area));
getAreaPostMovesV2(widerArea, moves.filter((_, i) => i > -1));
printArea(widerArea);
getGPSSumV2(widerArea);


####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################


9021